In [ ]:
# If you are using a MacBook with a M1 processor you have to make sure to set the docker default platform to linux/amd64
import os
os.environ["DOCKER_DEFAULT_PLATFORM"]="linux/amd64"

In [ ]:
# installation
!docker compose version >/dev/null
!docker info >/dev/null
!pip install -r ../requirements.txt

In [ ]:
# imports
from fondant.pipeline import ComponentOp, Pipeline
from pathlib import Path

from fondant.pipeline.compiler import DockerCompiler
from fondant.pipeline.runner import DockerRunner

Make sure to run the weaviate instance

In [ ]:
import weaviate
weaviate_client = weaviate.Client("http://localhost:8080")

In [ ]:
from iterative_eval_abstracts.main import IterativeEvaluation

In [ ]:
iter_eval = IterativeEvaluation(
    pipeline_dir="./data-dir",
    embed_model_provider="huggingface",
    embed_model="all-MiniLM-L6-v2",
    weaviate_url="http://192.168.64.1:8080",
    weaviate_class_name="Pipeline_1"
)

In [ ]:
index = iter_eval.run_indexing_pipeline(
    hf_dataset_name="wikitext@~parquet",
    data_column_name="text",
    n_rows_to_load=1000,
    chunk_size=512,
    chunk_overlap=32
)

In [ ]:
eval_results = iter_eval.run_evaluation_pipeline(
    csv_dataset_uri="/data/wikitext_1000_q.csv", #make sure it is the same as mounted file
    csv_column_separator=";",
    question_column_name="question",
    top_k=3,
    llm_name="OpenAI",
    llm_kwargs={"openai_api_key": ""},
    metrics=["context_precision", "context_relevancy"]
)

In [ ]:
# display aggregated results
eval_results

## Grid-Search (not yet developed)

In [ ]:
# grid-search/iterative loop
import itertools
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

# Define the values for grid search
chunk_sizes = [128, 256, 512]
chunk_overlaps = [10, 25, 50]
embed_models = [("huggingface","all-MiniLM-L6-v2"), ("huggingface""BAAI/bge-small-en"), ("huggingface""BAAI/bge-large-zh-v1.5")]
top_k = [2, 3, 5]

# Fixed parameters
hf_dataset_name = "wikitext@~parquet"
data_column_name = "text"
load_sample = True
weaviate_url = "http://host.docker.internal:8080"
csv_dataset_uri = "/data/wikitext_1000_q.csv"
csv_column_separator = ";"
question_column_name = "question"
llm_name = "OpenAI"
llm_kwargs = {"openai_api_key": ''} 
metrics = ["context_precision", "context_relevancy"]

# Results dictionary to store results for each iteration
results_dict = {}

# Perform grid search
for i, chunk_size, chunk_overlap, embed_model in enumerate(itertools.product(chunk_sizes, chunk_overlaps, embed_models, top_k), start=1):
    # Call the run_evaluation_pipeline function with the current parameter combination
    indexing_class_name = f"Index_{i}"
    logging.info(f"Running indexing for iteration {i} with chunk_size={chunk_size}, chunk_overlap={chunk_overlap}, embed_model={embed_model}")
    
    run_indexing_pipeline(
        hf_dataset_name=hf_dataset_name,
        data_column_name=data_column_name,
        load_sample=load_sample,
        chunk_size=chunk_sizes,
        chunk_overlap=chunk_overlaps,
        embed_model_provider=embed_models[0],
        embed_model=embed_models[1],
        weaviate_url=weaviate_url,
        weaviate_class_name=indexing_class_name
    )

    # Run evaluation pipeline
    evaluation_class_name = f"Index_{i}"
    logging.info(f"Running evaluation for iteration {i} with top_k={top_k}")

    results = run_evaluation_pipeline(
        csv_dataset_uri=csv_dataset_uri,
        csv_column_separator=csv_column_separator,
        question_column_name=question_column_name,
        embed_model_provider=embed_models[0],
        embed_model=embed_models[1],
        weaviate_url=weaviate_url,
        weaviate_class_name=indexing_class_name,
        top_k=top_k,  # Set your desired top_k value
        llm_name=llm_name,
        llm_kwargs=llm_kwargs
        metrics=metrics  # Set your desired metrics
    )

    # Save the results in the dictionary
    results_dict[(chunk_size, chunk_overlap, embed_model, top_k)] = results #results should be a dictionary with {"metric 1": x, "metric 2": y}

# Print or use the results as needed
print(results_dict)
